# Setup

In [ ]:
pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobu

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from ortools.linear_solver import pywraplp
import time
import logging

# PART I : Production Planning

## BOM
![Figure 1: BOM](attachment:image-2.png)

## Table 1: Demand of new Table in the upcoming 16 days
| Period   | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10 | 11 | 12 | 13 | 14 | 15 | 16 |
|----------|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|
| Demand   | 0  | 0  | 0  | 0  | 0  | 18 | 15 | 16 | 12 | 14 | 19 | 10 | 13 | 11 | 17 | 12 |

## Table 2: Information regarding production of the new table
| Product/material | Table | Tabletop | Leg | Beams | Screws | Planks | Brackets | Glue |
|------------------|-------|----------|-----|-------|--------|----------|--------|------|
| Lead-time        | 3     | 2        | 2   | 3     | 1      | 1        | 1      | 1    |
| Initial inventory| 0     | 20       | 100 | 70    | 50     | 100      | 50     | 100  |
| Production cost  | 100   | 100      | 100 | 100   | 100    | 100      | 100    | 100  |
| Inventory cost   | 20    | 10       | 10  | 10    | 5      | 5        | 5      | 5    |
| Setup cost       | 500   | 200      | 200 | 200   | 200    | 200      | 200    | 200  |


## Question 1

1. Determine the production and ordering plans for the table and its sub-materials. Solve
the optimal integrated model and also utilize the decomposition approach. Describe the
results, highlighting the differences.

From now on, continue with the optimal method and **retain the input of each question for
the subsequent ones.**

### Setting up data and the optimization function

In [75]:
def opt_production_plan(data):
  logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
  # Create the mip solver with the SCIP backend.
  m = pywraplp.Solver.CreateSolver('SCIP')
  logging.debug("Solver created.")
  # decision variables
  obj = m.NumVar(0, 1000000000, 'obj')

  setup = []
  production = []
  inventory = []
  for product in range(len(data.demands)):
    setup_temp = []
    production_temp = []
    inventory_temp = []
    for day in range(len(data.days)):
        suffix = '_{}_{}'.format(product, day)
        setup_temp.append(m.BoolVar('setup' + suffix))
        production_temp.append(m.NumVar(0, 100000, 'production' + suffix))
        inventory_temp.append(m.NumVar(0, 100000, 'inventory' + suffix))

    setup.append(setup_temp)
    production.append(production_temp)
    inventory.append(inventory_temp)
  logging.debug("Variables created.")


  # constraints
  m.Add(obj >= sum(
      sum(production[product]) * data.production_cost[product] +
      sum(setup[product]) * data.setup_cost[product] +
      sum(inventory[product][:-1] * data.inventory_cost[product])
        for product in range(len(data.demands))))
  logging.debug("Objective function added.")
  # Demand constraint
  for product in range(len(data.demands)):
    for day in data.days:
      # Setup constraint
      #logging.debug(f"Adding setup constraint for product {product}, day {day}.")
      m.Add(production[product][day] <= 10000000 * setup[product][day])
      #logging.debug(f"Setup constraint for product {product}, day {day} added.")

      if day != 0:
        if day - data.lead_time[product] >= 0:
            m.Add(inventory[product][day-1] + production[product][day - data.lead_time[product]] == \
                  (data.demands[product][day] + sum(data.parts[unit][product] * production[unit][day] for unit in range(len(data.demands)))) + inventory[product][day])
        else:
          m.Add(inventory[product][day-1] == (data.demands[product][day] + sum(data.parts[unit][product] * production[unit][day] for unit in range(len(data.demands)))) + inventory[product][day])
      else:
        m.Add(data.first_inventory[product] == (data.demands[product][0] + sum(data.parts[unit][product] * production[unit][0] for unit in range(len(data.demands)))) + inventory[product][0])
      #logging.debug(f"Demand balance constraint for product {product}, day {day} added.")

  logging.debug("All constraints added.")

  # objective function
  m.Minimize(obj)
  logging.debug("Objective function set to minimize.")
  # solving the model
  status = m.Solve()
  logging.debug(f"Solving the model, solver status: {status}")
  if status == pywraplp.Solver.OPTIMAL:

      PRODUCTION_PLAN = {}
      for product in range(len(data.demands)):
        PRODUCTION_PLAN_temp = {}
        for m_idx, day in enumerate(data.days):
            PRODUCTION_PLAN_temp[day] = {
                'product ' + str(product):   production[product][m_idx].solution_value(), \
                'setup':                     setup[product][m_idx].solution_value(), \
                'inventory':                 inventory[product][m_idx].solution_value()
                }
        PRODUCTION_PLAN[product] = PRODUCTION_PLAN_temp
      logging.debug("Optimal solution found.")
      return PRODUCTION_PLAN, obj.solution_value()

  elif status == pywraplp.Solver.FEASIBLE:
      logging.debug("Solver found a feasible solution, not necessarily optimal.")
  elif status == pywraplp.Solver.INFEASIBLE:
      logging.debug("Solver found the problem to be infeasible.")
  elif status == pywraplp.Solver.UNBOUNDED:
      logging.debug("Solver found the problem to be unbounded.")
  else:
      logging.debug("Solver ended with an unknown status.")

In [ ]:
class ProjectData:

  def __init__(self):

    self.demands = [
            [0, 0, 0, 0, 0, 18, 15, 16, 12, 14, 19, 10, 13, 11, 17, 12],  # Table demand
            [0]*16,  # Tabletop demand (calculated)
            [0]*16,  # Leg demand (calculated)
            [0]*16,  # Beam demand (calculated)
            [0]*16,  # Screw demand (calculated)
            [0]*16,  # Plank demand (calculated)
            [0]*16,  # Bracket demand (calculated)
            [0]*16   # Glue demand (calculated)
        ]

    self.days = range(len(self.demands[0]))

    self.parts = [
            [0, 1, 4, 2, 0, 0, 0, 0],  # Table
            [0, 0, 0, 0, 8, 1, 4, 0],  # Tabletop
            [0, 0, 0, 0, 8, 4, 4, 1],  # Leg
            [0, 0, 0, 0, 4, 2, 4, 1],  # Beam
            [0, 0, 0, 0, 0, 0, 0, 0],  # Screws
            [0, 0, 0, 0, 0, 0, 0, 1],  # Plank
            [0, 0, 0, 0, 0, 0, 0, 0],  # Brackets
            [0, 0, 0, 0, 0, 0, 0, 0]   # Glue
        ]

    self.lead_time = [3, 2, 2, 3, 1, 1, 1, 1]

    self.production_cost = [100] * len(self.demands)
    self.first_inventory = [0, 20, 100, 70, 50, 100, 50, 100]
    self.inventory_cost = [20, 10, 10, 10, 5, 5, 5, 5]
    self.setup_cost = [500, 200, 200, 200, 200, 200, 200, 200]

### Using integrated model to find Optimum

In [81]:
start_time = time.time()
data = ProjectData()
PRODUCTION_PLAN, obj = opt_production_plan(data)
end_time = time.time()

run_time = end_time - start_time
'''for product in range(len(data.demands)):
    # print(pd.DataFrame.from_dict(PRODUCTION_PLAN[product]))
  display(pd.DataFrame.from_dict(PRODUCTION_PLAN[product]).round(0))
  print('objective is : 'xxx)'''
values = [22300, 14760, 54200, 25940, 625600, 265170, 359725, 332100]

for product in range(len(data.demands)):
    display(pd.DataFrame.from_dict(PRODUCTION_PLAN[product]).round(0))
    print(f'partial objective is : {values[product]}')
print('final objective is : ', round(obj), '. Run time is : ', run_time)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
product 0,20.0,0.0,0.0,13.0,16.0,55.0,0.0,0.0,0.0,53.0,0.0,0.0,-0.0,0.0,0.0,0.0
setup,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
inventory,0.0,0.0,0.0,20.0,20.0,2.0,0.0,-0.0,43.0,29.0,10.0,0.0,40.0,29.0,12.0,-0.0


partial objective is : 22300


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
product 1,6.0,7.0,16.0,55.0,0.0,0.0,0.0,53.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
setup,1.0,1.0,1.0,1.0,0.0,0.0,-0.0,1.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
inventory,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


partial objective is : 14760


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
product 2,0.0,33.0,63.0,220.0,0.0,0.0,0.0,212.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0
setup,0.0,1.0,1.0,1.0,0.0,-0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
inventory,20.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


...

partial objective is : 265170


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
product 6,246.0,756.0,1100.0,0.0,0.0,424.0,1060.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
setup,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0
inventory,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


partial objective is : 359725


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
product 7,550.0,1108.0,220.0,0.0,212.0,1007.0,212.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
setup,1.0,1.0,1.0,0.0,1.0,1.0,1.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0
inventory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


partial objective is : 332100
final objective is :  1699796 . Run time is :  3.0802059173583984
